In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from transformers import BertTokenizer
import torch
from torch import nn
from transformers import BertModel
import numpy as np
from torch.optim import Adam
from tqdm import tqdm

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


import os

df = pd.read_csv("/kaggle/input/feedback-prize-2021/train.csv")
df

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))

In [ ]:
df_new = df
from gensim.utils import simple_preprocess
# Tokenize the text column to get the new column 'tokenized_text'
df_new['text_without_stopwords'] = df_new['discourse_text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (STOPWORDS)]))
df_new['tokenized_text'] = [simple_preprocess(line, deacc=True) for line in df_new['text_without_stopwords']] 
print(df_new['tokenized_text'].head(10))

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df_new, test_size = 0.2, stratify = df_new['discourse_type'])

In [ ]:
train_text = train['tokenized_text']
train_lab = train['discourse_type']
test_text = test['tokenized_text']
test_lab = test['discourse_type']

In [ ]:
from collections import Counter
results = Counter()
df['text_without_stopwords'].str.lower().str.split().apply(results.update)
len(results.values())

In [ ]:
vocab_size = 90000
embedding_dim = 64
max_length = 100
trunc_type = 'post'
padding_type = 'post'
oov_tok = '<OOV>'

In [ ]:
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_text)
word_index = tokenizer.word_index

In [ ]:
train_sequences = tokenizer.texts_to_sequences(train_text)

In [ ]:
train_padded = pad_sequences(train_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [ ]:
w = train['tokenized_text']
l = []
for i in w:
    l.append(len(i))
import statistics
statistics.mean(l)

In [ ]:
test_sequences = tokenizer.texts_to_sequences(test_text)
test_padded = pad_sequences(test_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [ ]:
label_tokenizer = Tokenizer()
label_tokenizer.fit_on_texts(df_new['discourse_type'].str.replace(' ', ''))

training_label_seq = np.array(label_tokenizer.texts_to_sequences(train_lab.str.replace(' ', '')))
test_label_seq = np.array(label_tokenizer.texts_to_sequences(test_lab.str.replace(' ', '')))

In [ ]:
train_label_final = training_label_seq - 1
test_label_final = test_label_seq - 1

In [ ]:
np.unique(train_label_final)

In [ ]:
import random
def reset_seeds():
    np.random.seed(123) 
    random.seed(123)
    tf.random.set_seed(123)

reset_seeds()

model = tf.keras.Sequential([
    # Add an Embedding layer expecting input vocab of size 5000, and output embedding dimension of size 64 we set at the top
    tf.keras.layers.Embedding(vocab_size, embedding_dim),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(embedding_dim)),
#    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    # use ReLU in place of tanh function since they are very good alternatives of each other.
    tf.keras.layers.Dense(embedding_dim, activation='relu')
    # When we have multiple outputs, softmax convert outputs layers into a probability distribution.
    tf.keras.layers.Dense(7, activation='softmax')
])
model.summary()

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
num_epochs = 3
history = model.fit(train_padded, train_label_final, epochs=num_epochs, validation_data=(test_padded, test_label_final), verbose=2)